In [1]:
import csv
import itertools
import pandas
import re

In [2]:
def plate_to_dict(file):
    '''Converts VersaMax Plate Reader text file to a dictionary
    
    keys equal plate reader rows a-h
    values are lists corresponding to columns 1-12 in each row
    '''
    with(open(file)) as f:
        plate = csv.reader(f, delimiter = '\t')
        alph = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        plate_dict = {}
        for i in itertools.islice(plate, 3, 11): 
            #96 well plate data is only stored in rows 3-11
            for z in alph:
                if z in plate_dict.keys():
                    next
                else: 
                    plate_dict[z] = i[2:14]
                    #VersaMax text files begin and end each row with spaces
                    #Only columns 2-14 correspond to actual data
                    break
    return plate_dict

In [3]:
def bind_dict_to_sample(input_file, 
                        input_template,
                        output_filename = 'untitled.csv',
                        samp_numrep = 2,
                        stan_numrep = 3,
                        blank_numrep = 3):
    
    '''Binds values from dictionary produced with plate_to_dict to
    appropriate sample IDs in a template data_frame provided by the 
    user. Template must include a column titled type for each sample
    filled with Sample, Standard, or Blank.  Template must also include
    a column titled position that includes a 96 well plate row ID
    followed by a 96 well plate column ID in the format A, 10 for 
    example.
    
    Note: Assumes technical replicates on plate are horizontal.
    
    input_file = VersaMax Plate Reader Text File
    input_template = user created csv file containing sample information, 
    and starting well location on 96 well plate.
    samp_numrep = technical replicates for each sample labeled Sample
    stan_numrep = technical replicates for each sample labeled Standard
    blank_numrep = technical replicates for each sample labeled Blank
    '''

    data = plate_to_dict(input_file)
    input_template = pandas.read_csv(input_template)
    
    #creates new columns labeled OD_1 - OD_(max number of replicates). Fills with 'None'
    z = max(samp_numrep, stan_numrep, blank_numrep)
    for j in range(1, z + 1):
        newcol = "OD_" + str(j)
        input_template[newcol] = None

    #Binds correct values from plate_to_dict dictionary to each sample based on position column    
    for index, row in input_template.iterrows():
        c = re.search(r'\w', row['position']) #row ID
        c = str.lower(c.group())
        d = re.search(r'\d+', row['position']) #column ID
        d = d.group()
        
        #collects sequential data from dictionary key starting from c: d based on number of replicates
        if row['type'] == 'Sample':
            values = list(data[c][int(d) - 1 : int(d) + samp_numrep - 1]) 
            od_match = list(range(1, len(values) + 1))
            for n in od_match:
                input_template.loc[index, 'OD_' + str(n)] = float(values[n - 1])
        elif row['type'] == 'Standard':
            values = list(data[c][int(d) - 1 : int(d) + stan_numrep - 1])
            od_match = list(range(1, len(values) + 1))
            for n in od_match:
                input_template.loc[index, 'OD_' + str(n)] = float(values[n - 1])
        elif row['type'] == 'Blank':
            values = list(data[c][int(d) - 1 : int(d) + blank_numrep - 1])
            od_match = list(range(1, len(values) + 1))
            for n in od_match:
                input_template.loc[index, 'OD_' + str(n)] = float(values[n - 1])
        else:
            print('Error: Unrecognized label in column "type"') #in case there is a typo in template 

    input_template.to_csv(output_filename)

In [4]:
bind_dict_to_sample('output_txt_file.txt', 'automated_ELISA_template.csv', 'post_python_file.csv', 2, 3, 3)